<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# Lab: Comparing Random Forest and XGBoost modeling performance
Estimated time needed: **20** minutes
    

## Objectives


After completing this lab, you will be able to:

* Use scikit-learn to implement Random Forest and XGBoost regression models
* Compare the performances of the two models 



## Introduction
In this lab, you'll create and measure the relative performances of Random Forest and XGBoost regression models for predicting house prices using the California Housing Dataset.
'Performance' means both speed and accuracy.


First, we need to install the libraries that will be required in this lab.


In [5]:
!pip install numpy==2.2.0
!pip install scikit-learn==1.6.0
!pip install matplotlib==3.9.3
!pip install xgboost==2.1.3

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ----------- ---------------------------- 3.7/12.6 MB 27.3 MB/s eta 0:00:01
   ------------------------------------- -- 11.8/12.6 MB 35.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 29.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
contourpy 1.2.0 requires numpy<2.0,>=1.20, but you have numpy 2.2.0 which is incompatible.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.0 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.0 which is incompatible.


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   -------------------------- ------------- 7.3/11.1 MB 45.4 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 40.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.5.1
    Uninstalling scikit-learn-1.5.1:
      Successfully uninstalled scikit-learn-1.5.1
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/7.8 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.8 MB 1.9 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/7.8 MB 2.1 MB/s eta 0:00:04
   -------- ------------------------------- 1.6/7.8 MB 2.2 MB/s eta 0:00:03
   ------------ --------------------------- 2.4/7.8 MB 2.4 MB/s eta 0:00:03
   -------------- ------------------------- 2.9/7.8 MB 2.6 MB/s eta 0:00:02
   ------------------ --------------------- 3.7/7.8 MB 2.7 MB/s eta 0:00:02
   

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'c:\\users\\louis\\anaconda3\\lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans.ttf'
Consider using the `--user` option or check the permissions.



   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   -- ------------------------------------- 6.6/124.9 MB 50.3 MB/s eta 0:00:03
   -- ------------------------------------- 7.3/124.9 MB 34.9 MB/s eta 0:00:04
   -- ------------------------------------- 8.7/124.9 MB 14.9 MB/s eta 0:00:08
   --- ------------------------------------ 9.7/124.9 MB 13.4 MB/s eta 0:00:09
   --- ------------------------------------ 9.7/124.9 MB 13.4 MB/s eta 0:00:09
   --- ------------------------------------ 9.7/124.9 MB 13.4 MB/s eta 0:00:09
   --- ------------------------------------ 9.7/124.9 MB 13.4 MB/s eta 0:00:09
   --- ------------------------------------ 10.2/124.9 MB 6.1 MB/s eta 0:00:19
   --- ------------------------------------ 10.2/124.9 MB 6.1 MB/s eta 0:00:19
   --- ------------------------------------ 10.2/124.9 MB 6.1 MB/s eta 0:00:19
   ---- ----------------------------------- 13.9/124.9 MB 6.0 MB/s eta 0:00:19
   ---- ----------------------------------- 14.2/124.9 MB 5.

You may now start by importing the required libraries:


In [39]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score
import time

In [8]:
# Load the California Housing dataset
data = fetch_california_housing()
X, y = data.data, data.target

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Exercise 1: How many observations and features does the dataset have?


In [14]:
# Enter your code here
N_observations, N_features = X.shape

print('Number of Observations: ' + str(N_observations))
print('Number of Features: ' + str(N_features))


Number of Observations: 20640
Number of Features: 8


<details><summary>Click here for the solution</summary>

```python

N_observations, N_features = X.shape
print('Number of Observations: ' + str(N_observations))
print('Number of Features: ' + str(N_features))


```

</details>


This is a moderately sized dataset used for this analysis.  
Keep in mind you are only using one dataset so you have to consider that the comparison may change with scale.


### Initialize models
In this step you define the number of base estimators, or individual trees, to be used in each model, and then intialize models for Random Forest regression and XGBoost regression.  You'll just use the default parameters to make the performance comparisons. As a part of the performance comparison, we'll also measure the training times for both models.


In [16]:
# Initialize models
n_estimators=100
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
xgb = XGBRegressor(n_estimators=n_estimators, random_state=42)

In [18]:
# Fit models
# Measure training time for Random Forest
start_time_rf = time.time()
rf.fit(X_train, y_train)
end_time_rf = time.time()
rf_train_time = end_time_rf - start_time_rf

# Measure training time for XGBoost
start_time_xgb = time.time()
xgb.fit(X_train, y_train)
end_time_xgb = time.time()
xgb_train_time = end_time_xgb - start_time_xgb

### Exercise 2. Use the fitted models to make predictions on the test set. 
Also, measure the time it takes for each model to make its predictions using the time.time() function to measure the times before and after each model prediction.


In [20]:
# Enter your code here

# Measure prediction time for Random Forest
start_time_rf = time.time()
y_pred_rf = rf.predict(X_test)
end_time_rf = time.time()
rf_pred_time = end_time_rf - start_time_rf

# Measure prediction time for XGBoost
start_time_xgb = time.time()
y_pred_xgb = xgb.predict(X_test)
end_time_xgb = time.time()
xgb_pred_time = end_time_xgb - start_time_xgb


<details><summary>Click here for the solution</summary>

```python
# Measure prediction time for Random Forest
start_time_rf = time.time()
y_pred_rf = rf.predict(X_test)
end_time_rf = time.time()
rf_pred_time = end_time_rf - start_time_rf

# Measure prediciton time for XGBoost
start_time_xgb = time.time()
y_pred_xgb = xgb.predict(X_test)
end_time_xgb = time.time()
xgb_pred_time = end_time_xgb - start_time_xgb

```

</details>


### Exercise 3:  Calulate the MSE and R^2 values for both models


In [22]:
# Enter your code here

mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_rf = r2_score(y_test, y_pred_rf)
r2_xgb = r2_score(y_test, y_pred_xgb)

<details><summary>Click here for the solution</summary>

```python
mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_rf = r2_score(y_test, y_pred_rf)
r2_xgb = r2_score(y_test, y_pred_xgb)
```

</details>


### Exercise 4:  Print the MSE and R^2 values for both models


In [26]:
# Enter your code here

print(f'Random Forest:  MSE = {mse_rf:.4f}, R^2 = {r2_rf:.4f}')
print(f'      XGBoost:  MSE = {mse_xgb:.4f}, R^2 = {r2_xgb:.4f}')


Random Forest:  MSE = 0.2554, R^2 = 0.8051
      XGBoost:  MSE = 0.2226, R^2 = 0.8301


<details><summary>Click here for the solution</summary>

```python

print(f'Random Forest:  MSE = {mse_rf:.4f}, R^2 = {r2_rf:.4f}')
print(f'      XGBoost:  MSE = {mse_xgb:.4f}, R^2 = {r2_xgb:.4f}')
```

</details>


You can see from the MSE and R^2 values that XGBoost is better than Random Forest, but the differences aren't overwhelming.


### Exercise 5:  Print the timings for each model


In [29]:
print(f'Random Forest:  Training Time = {rf_train_time:.3f} seconds, Testing time = {rf_pred_time:.3f} seconds')
print(f'      XGBoost:  Training Time = {xgb_train_time:.3f} seconds, Testing time = {xgb_pred_time:.3f} seconds')

Random Forest:  Training Time = 13.572 seconds, Testing time = 0.159 seconds
      XGBoost:  Training Time = 0.228 seconds, Testing time = 0.005 seconds


<details><summary>Click here for the solution</summary>

```python
print(f'Random Forest:  Training Time = {rf_train_time:.3f} seconds, Testing time = {rf_pred_time:.3f} seconds')
print(f'      XGBoost:  Training Time = {xgb_train_time:.3f} seconds, Testing time = {xgb_pred_time:.3f} seconds')
```

</details>


What is very impressive is the difference in computation time between XGBoost and Random Forest for both training and testing.


Next, you want to generate scatter plots between the predicted and actual values for both models so you can visually evaluate how well each model performs.
We'll also plot lines one standard deviation of the test data above and below the ideal line, that is, the line that represents the perfect regressor, where the predictions are all correct.


### Exercise 6. Calculate the standard deviation of the test data


In [31]:
# Enter your code here
std_y = np.std(y_test)

<details><summary>Click here for the solution</summary>

``` python
# Standard deviation of y_test
std_y = np.std(y_test)
```

</details>


In [33]:
start_time_rf = time.time()
y_pred_rf = rf.predict(X_test)
end_time_rf = time.time()
rf_pred_time = end_time_rf - start_time_rf

# Measure prediciton time for XGBoost
start_time_xgb = time.time()
y_pred_xgb = xgb.predict(X_test)
end_time_xgb = time.time()
xgb_pred_time = end_time_xgb - start_time_xgb

mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_rf = r2_score(y_test, y_pred_rf)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f'Random Forest:  MSE = {mse_rf:.4f}, R^2 = {r2_rf:.4f}')
print(f'      XGBoost:  MSE = {mse_xgb:.4f}, R^2 = {r2_xgb:.4f}')
print(f'Random Forest:  Training Time = {rf_train_time:.3f} seconds, Testing time = {rf_pred_time:.3f} seconds')
print(f'      XGBoost:  Training Time = {xgb_train_time:.3f} seconds, Testing time = {xgb_pred_time:.3f} seconds')
std_y = np.std(y_test)

Random Forest:  MSE = 0.2554, R^2 = 0.8051
      XGBoost:  MSE = 0.2226, R^2 = 0.8301
Random Forest:  Training Time = 13.572 seconds, Testing time = 0.127 seconds
      XGBoost:  Training Time = 0.228 seconds, Testing time = 0.002 seconds


### Visualize the results


In [55]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

# Random Forest plot
ax = axes[0]
ax.scatter(y_test, y_pred_rf, alpha=0.5, color="blue", edgecolors='k', label="Predictions")
ax.axline((0, 0), slope=1, color="k", linestyle="--", linewidth=2, label="Perfect Model")
ax.axline((0, std_y), slope=1, color="r", linestyle="--", linewidth=1, label="+/-1 Std Dev")
ax.axline((0, -std_y), slope=1, color="r", linestyle="--", linewidth=1)
ax.set_ylim(0, 6)
ax.set_title("Random Forest Predictions vs Actual")
ax.set_xlabel("Actual Values")
ax.set_ylabel("Predicted Values")
ax.legend()

# XGBoost plot
ax = axes[1]
ax.scatter(y_test, y_pred_xgb, alpha=0.5, color="orange", edgecolors='k', label="Predictions")
ax.axline((0, 0), slope=1, color="k", linestyle="--", linewidth=2, label="Perfect Model")
ax.axline((0, std_y), slope=1, color="r", linestyle="--", linewidth=1, label="+/-1 Std Dev")
ax.axline((0, -std_y), slope=1, color="r", linestyle="--", linewidth=1)
ax.set_ylim(0, 6)
ax.set_title("XGBoost Predictions vs Actual")
ax.set_xlabel("Actual Values")
ax.legend()

plt.tight_layout()
plt.show()


ImportError: cannot import name 'LoadFlags' from 'matplotlib.ft2font' (C:\Users\louis\anaconda3\Lib\site-packages\matplotlib\ft2font.cp312-win_amd64.pyd)

In [51]:
!pip uninstall matplotlib
!pip install matplotlib

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ----------- ---------------------------- 2.4/8.0 MB 26.5 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 35.5 MB/s eta 0:00:00


ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\louis\\anaconda3\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\ttf\\DejaVuSans.ttf'
Consider using the `--user` option or check the permissions.



Both models performed very well. Most of their predictions fall within a standard deviation of the target. Interestingly, random forest "respects" the upper bound (the maximum value) present in the target by staying within its limits, while XGBoost "overshoots", or exceeds this limit. 


### Congratulations! You're ready to move on to your next lesson.

## Author

<a href="https://www.linkedin.com/in/jpgrossman/" target="_blank">Jeff Grossman</a>

### Other Contributors

<a href="https://www.linkedin.com/in/abhishek-gagneja-23051987/" target="_blank">Abhishek Gagneja</a>

<!-- ## Changelog

| Date | Version | Changed by | Change Description |
|:------------|:------|:------------------|:---------------------------------------|
| 2024-11-05 | 1.0  | Jeff Grossman    | Create content | -->



## <h3 align="center"> © IBM Corporation. All rights reserved. <h3/>
